In [1]:
#! /usr/bin/env python
# coding=utf-8
#================================================================
#   Copyright (C) 2019 * Ltd. All rights reserved.
#
#   Editor      : VIM
#   File name   : image_demo.py
#   Author      : YunYang1994
#   Created date: 2019-01-20 16:06:06
#   Description :
#
#================================================================

import cv2
import numpy as np
import core.utils as utils
import tensorflow as tf
import os
import pandas as pd
import csv
from PIL import Image
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
%matplotlib inline
return_elements = ["input/input_data:0", "pred_sbbox/concat_2:0", "pred_mbbox/concat_2:0", "pred_lbbox/concat_2:0"]
pb_file         = "./yolov3.pb"
image_path      = "../aidea/test_images/"
num_classes     = 13
input_size      = 608
graph           = tf.Graph()
result = []
for i in os.listdir(image_path):
    print(i)
    original_image = cv2.imread(image_path + i)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    original_image_size = original_image.shape[:2]
    image_data = utils.image_preporcess(np.copy(original_image), [input_size, input_size])
    image_data = image_data[np.newaxis, ...]

    return_tensors = utils.read_pb_return_tensors(graph, pb_file, return_elements)


    with tf.Session(graph=graph) as sess:
        pred_sbbox, pred_mbbox, pred_lbbox = sess.run(
            [return_tensors[1], return_tensors[2], return_tensors[3]],
                    feed_dict={ return_tensors[0]: image_data})

    pred_bbox = np.concatenate([np.reshape(pred_sbbox, (-1, 5 + num_classes)),
                                np.reshape(pred_mbbox, (-1, 5 + num_classes)),
                                np.reshape(pred_lbbox, (-1, 5 + num_classes))], axis=0)

    bboxes = utils.postprocess_boxes(pred_bbox, original_image_size, input_size, 0.25)
    bboxes = utils.nms(bboxes, 0.37, method='nms')
    image = utils.draw_bbox(original_image, bboxes)
    image = Image.fromarray(image)
    image.save('./test/' + i)
    for k in bboxes:
        temp = []
        temp.append(i)
        temp.append(k[5])
        temp.append(k[0])
        temp.append(k[1])
        temp.append(abs(k[2] - k[0]))
        temp.append(abs(k[3] - k[1]))
        temp.append(k[4])
        if os.path.isfile('submission.csv'):
            with open('submission.csv', 'a') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(temp)
        else:
            with open('submission.csv', 'w') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(temp)




200811058.jpg
Instructions for updating:
Use tf.gfile.GFile.
2008112313.jpg
200812025.jpg
200811257.jpg
20081105.jpg
200811224.jpg
200811265.jpg
2008112325.jpg
2008112334.jpg
20080508.jpg
200811184.jpg
20081130.jpg
20080905.jpg
200812037.jpg
2008112336.jpg
2008112934.jpg
2008113039.jpg
200811294.jpg
200811266.jpg
2008113032.jpg
200811296.jpg
200811292.jpg
200801221.jpg
200808213.jpg
200812035.jpg
